In [1]:

import time
import pathlib
import sys
sys.path.insert(0, str(pathlib.Path("..").absolute()))

from computation_sim.nodes import (
    ConstantNormalizer,
    FilteringMISONode,
    Node,
    OutputNode,
    PeriodicEpochSensor,
    RingBufferNode,
    SinkNode,
    SourceNode,
)
from computation_sim.system import Action, System, SystemDrawer
from computation_sim.time import Clock, FixedDuration
from IPython.display import display, clear_output

In [2]:
clock = Clock(0)

# Normalizer for time
normalizer = ConstantNormalizer(100.0)

# Setup the source
sensor = PeriodicEpochSensor(0, 100, FixedDuration(0))
source_node_0 = SourceNode(clock.as_readonly(), sensor, "SOURCE_NODE_0")
sensor = PeriodicEpochSensor(0, 100, FixedDuration(0))
source_node_1 = SourceNode(clock.as_readonly(), sensor, "SOURCE_NODE_1")

# Setup sink and output
output_node = OutputNode(clock.as_readonly(), id="OUTPUT_NODE", age_normalizer=normalizer)
lost_node = SinkNode(clock.as_readonly(), id="LOST_MESSAGES")

# Setup buffers
buffer_0 = RingBufferNode(
    clock.as_readonly(),
    "BUFFER_0",
    overflow_cb=lambda x: lost_node.receive(x),
    age_normalizer=normalizer,
)
buffer_1 = RingBufferNode(
    clock.as_readonly(),
    "BUFFER_1",
    overflow_cb=lambda x: lost_node.receive(x),
    age_normalizer=normalizer,
)

# Setup the processor node
compute_node = FilteringMISONode(
    clock.as_readonly(),
    FixedDuration(50),
    id="COMPUTE_NODE",
    age_normalizer=normalizer,
)

# Connect outputs
source_node_0.add_output(buffer_0)
source_node_1.add_output(buffer_1)
buffer_0.add_output(compute_node)
buffer_1.add_output(compute_node)
compute_node.set_output_fail(lost_node)
compute_node.set_output_pass(output_node)

# Set-up the action
compute_action = Action(name="ACT_COMPUTE_NODE")
compute_action.register_callback(buffer_0.trigger, 1)
compute_action.register_callback(buffer_1.trigger, 1)
compute_action.register_callback(compute_node.trigger, 0)

system = System()
system.add_action(compute_action)
system.add_node(lost_node)
system.add_node(output_node)
system.add_node(compute_node)
system.add_node(source_node_0)
system.add_node(source_node_1)
system.add_node(buffer_0)
system.add_node(buffer_1)

In [3]:

clock.reset()
system.reset()
system.update()
drawer = SystemDrawer()
drawer.build(system._node_graph)
clear_output(wait=True)
display(drawer.fw)
    
while clock.get_time() < 10000:
    print(clock.get_time())
    system.update()
    if (clock.get_time() + 10) % 100 == 0:
        system.act([1])
    drawer.update(system._node_graph)
    clear_output(wait=True)
    display(drawer.fw)
    clock += 10
    time.sleep(0.1)


        

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': '#888', 'width': 0.5},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'uid': '46e5b8da-abaa-4108-bea5-cf5fbffa4afc',
              'x': [0.3636363636363636, 1.0, None, 0.3636363636363636, 1.0, None,
                    -0.9090909090909091, -0.27272727272727276, None,
                    -0.27272727272727276, 0.3636363636363636, None,
                    -0.9090909090909091, -0.27272727272727276, None,
                    -0.27272727272727276, 0.3636363636363636, None],
              'y': [0.0, -0.3181818181818182, None, 0.0, 0.3181818181818182, None,
                    -0.3181818181818182, -0.3181818181818182, None,
                    -0.3181818181818182, 0.0, None, 0.3181818181818182,
                    0.3181818181818182, None, 0.3181818181818182, 0.0, None]},
             {'hovertext': [num_messages = 0, is_occupied = 1.0<br>msg.age_o

KeyboardInterrupt: 